In [1]:
import scrapy
from scrapy.item import Item, Field
from scrapy.loader import ItemLoader
from scrapy.crawler import CrawlerRunner
from twisted.internet import reactor

In [2]:
class ArticleItem(Item):
    date = Field()
    url = Field()
    title = Field()
    desc = Field()
    author = Field()

In [3]:
class ArticleSpider(scrapy.Spider):
    name = 'article'
    
    def start_requests(self):
        urls = ['https://www.nytimes.com/section/world',]
        
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)
        
    
    def parse(self, response):
        load = ItemLoader(item=ArticleItem(), response=response)
        recent_articles = response.xpath('//section[@id="stream-panel"]//ol/li/div')
        
        for article in recent_articles:
            content = article.xpath('div[1]/a')
            
            load.add_xpath('date', article.xpath('div[2]/span/text()').get())
            load.add_xpath('url', 'https://www.nytimes.com/' + content.xpath('@href').get())
            load.add_xpath('title', content.xpath('h2/text()').get())
            load.add_xpath('desc', content.xpath('p/text()').get())
            load.add_xpath('author', content.xpath('//p[contains(text(), "By")]/span/text()').get())
            
            yield load.load_item()

In [4]:
runner = CrawlerRunner(settings={
    'FEED_EXPORT_ENCODING': 'utf-8',
    'FEEDS': {'sample.json': {'format': 'json'}}
})
d = runner.crawl(ArticleSpider)
d.addBoth(lambda _: reactor.stop())

/home/rimov/anaconda3/envs/proj_happenings/lib/python3.10/site-packages/scrapy/utils/request.py:231: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on how to handle this deprecation.
  return cls(crawler)


In [5]:
reactor.run()